In [1]:
import numpy as np

# ======= ompl
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og
import copy

In [2]:
class MyStateSpace(ob.RealVectorStateSpace):
    def __init__(self, ndof):
        self.ndof = ndof
        super(MyStateSpace, self).__init__(self.ndof)

        lower_limits = [-1.7016, -2.147, -3.0541, -0.05, -3.059, -1.5707, -3.509]
        upper_limits = [1.7016, 1.047, 3.0541, 2.618, 3.059, 2.094, 3.509]

        joint_bounds = ob.RealVectorBounds(self.ndof)
        for i in range(self.ndof):
            joint_bounds.setLow(i, lower_limits[i])
            joint_bounds.setHigh(i, upper_limits[i])

        self.setBounds(joint_bounds)
        self.setup()

In [3]:
class MyStateValidityChecker(ob.StateValidityChecker):
    def __init__(self, space_information):
        super(MyStateValidityChecker, self).__init__(space_information)
        self.space_information = space_information

    def isValid(self, state):
        return self.space_information.satisfiesBounds(state)

In [4]:
class MyGoalRegion(ob.Goal):
    def __init__(self, si, goal_config, ndof):
        super(MyGoalRegion, self).__init__(si)
        # self.setThreshold(0.1)
        self.goal_config = goal_config
        self.ndof = ndof

    
        
    
    def isSatisfied(self, state):
        state_vector = [state[i] for i in range(self.ndof)]
        goal_state_vector = [self.goal_config[i] for i in range(self.ndof)]
        joints_diff = [abs(a - b) for a, b in zip(goal_state_vector, state_vector)]
        dis = np.asarray(joints_diff).sum()

        
        
        return dis < 0.1

In [5]:
class MyRRT:
    def __init__(self, ndof, iksolver, step_size=0.05):
        self.ndof = ndof
        self.iksolver = iksolver
        self.state_space = MyStateSpace(ndof)
        # self.control_space = MyControlSpace(self.state_space, ndof)
        self.simple_setup = og.SimpleSetup(self.state_space)
        si = self.simple_setup.getSpaceInformation()
        # si.setPropagationStepSize(step_size)
        # si.setMinMaxControlDuration(1, 1)
        # si.setDirectedControlSamplerAllocator(oc.DirectedControlSamplerAllocator(directedControlSamplerAllocator))

        # propagator = MyStatePropagator(self.simple_setup.getSpaceInformation(), ndof)
        # self.simple_setup.setStatePropagator(propagator)

        

        # self.planner = oc.KPIECE1(self.simple_setup.getSpaceInformation())
        # self.planner.setup()
        # ========= RRT planner ============
        vc = MyStateValidityChecker(self.simple_setup.getSpaceInformation())
        self.simple_setup.setStateValidityChecker(vc)

        self.planner = og.RRTstar(self.simple_setup.getSpaceInformation())
        p_goal = 0.05
        self.planner.setGoalBias(p_goal)
        self.planner.setRange(0.05)


        self.simple_setup.setPlanner(self.planner)
        self.simple_setup.setup()
        
        


    def psolve(self, start, goal, timeout):
        self.simple_setup.setStartState(start)
        mygoalregion = MyGoalRegion(self.simple_setup.getSpaceInformation(), goal, self.ndof)
        self.simple_setup.setGoalState(goal)

        
        
        # print(self.simple_setup)
        # print("Setup Done")
        

        if self.simple_setup.solve(timeout):
            print("Solved")
            if self.simple_setup.haveExactSolutionPath():
                print ("Exact Solution.")
                return self.simple_setup.getSolutionPath()
            elif self.simple_setup.haveSolutionPath():
                print ("Approximate Solution.")
                return self.simple_setup.getSolutionPath()
        else:
            print ("No Solution Found.")
            return None

In [6]:

if __name__ == '__main__':

    # rospy.init_node('baxter_planning')

    ndof = 7

    planner = MyRRT(ndof, 0.1)

    start = ob.State(planner.state_space)
    goal = ob.State(planner.state_space)

    start_vector = np.array([0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0])
    goal_vector = np.array([0.3, 0.5, 0.0, 1.0, 0.0, 0.0, 0.0])
    for i in range(ndof):
        start[i] = start_vector[i]
        goal[i] = goal_vector[i]

    # ============== plan ===============
    print("REturned")
    result_path = planner.psolve(start, goal, 10)
    path = []
    joints = np.empty(ndof)
    for i in range(result_path.getStateCount()):
        
        for j in range(ndof):
            joints[j] = result_path.getStates()[i][j]
        path.append(copy.deepcopy(joints))

    print(path)


REturned
Info:    RRTstar: No optimization objective specified. Defaulting to optimizing path length for the allowed planning time.
Info:    RRTstar: Started planning with 1 states. Seeking a solution better than 0.00000.
Info:    RRTstar: Initial k-nearest value of 607
Info:    RRTstar: Found an initial solution with a cost of 0.94 in 318 iterations (319 vertices in the graph)
SolvedInfo:    RRTstar: Created 4562 new states. Checked 10408203 rewire options. 1 goal states in tree. Final solution cost 0.936
Info:    Solution found in 10.037952 seconds

Exact Solution.
[array([0. , 0. , 0. , 0.5, 0. , 0. , 0. ]), array([-0.00716732, -0.00175512,  0.01382953,  0.51996449,  0.03081214,
       -0.00538555, -0.02961958]), array([ 0.01302374,  0.0312268 ,  0.01292047,  0.55151872,  0.02878677,
       -0.00503154, -0.0276726 ]), array([ 0.02408061,  0.02708876, -0.02350956,  0.55598067,  0.02188944,
        0.00513687, -0.05704259]), array([ 0.04356687,  0.06048719, -0.02184924,  0.58733867,  

In [7]:
class SphereConstraint(ob.Constraint):

    def __init__(self):
        super(SphereConstraint, self).__init__(3, 1)

    def function(self, x, out):
        out[0] = np.linalg.norm(x) - 1

    def jacobian(self, x, out):
        nrm = np.linalg.norm(x)
        if np.isfinite(nrm) and nrm > 0:
            out[0, :] = x / nrm
        else:
            out[0, :] = [1, 0, 0]


In [8]:
def obstacles(x):
    if x[2] > -0.8 and x[2] < -0.6:
        if x[1] > -0.05 and x[1] < 0.05:
            return x[0] > 0
        return False
    elif x[2] > -0.1 and x[2] < 0.1:
        if x[0] > -0.05 and x[0] < 0.05:
            return x[1] < 0
        return False
    elif x[2] > 0.6 and x[2] < 0.8:
        if x[1] > -0.05 and x[1] < 0.05:
            return x[0] < 0
        return False
    return True

In [9]:
rvss = ob.RealVectorStateSpace(3)
bounds = ob.RealVectorBounds(3)
bounds.setLow(-2)
bounds.setHigh(2)
rvss.setBounds(bounds)

# Create our constraint.
constraint = SphereConstraint()

css = ob.ProjectedStateSpace(rvss, constraint)
csi = ob.ConstrainedSpaceInformation(css)
css.setup()

ss = og.SimpleSetup(csi)



In [10]:
start = ob.State(css)
goal = ob.State(css)
start[0] = 0
start[1] = 0
start[2] = -1
goal[0] = 0
goal[1] = 0
goal[2] = 1
ss.setStartAndGoalStates(start, goal)

ss.setStateValidityChecker(ob.StateValidityCheckerFn(obstacles))

ss.setPlanner(og.RRTstar(csi))

In [11]:
ss.setup()
stat = ss.solve(10)
print(ss)

if stat:
    print("Solution Found")
    path = ss.getSolutionPath()
    print(path.printAsMatrix())

Debug:   RRTstar: Planner range detected to be 1.385641
Info:    RRTstar: No optimization objective specified. Defaulting to optimizing path length for the allowed planning time.
Info:    RRTstar: Started planning with 1 states. Seeking a solution better than 0.00000.
Info:    RRTstar: Initial k-nearest value of 45
Info:    ProblemDefinition: Adding approximate solution from planner RRTstar
Info:    RRTstar: Created 377 new states. Checked 71253 rewire options. 0 goal states in tree. Final solution cost inf
Info:    Solution found in 10.126195 seconds
Properties of the state space 'Space2'
  - signature: 2 1 3 
  - dimension: 3
  - extent: 6.9282
  - sanity checks for state space passed
  - probability of valid states: 0.654
  - average length of a valid motion: 0.305869
  - average number of samples drawn per second: sampleUniform()=29941.6 sampleUniformNear()=27981.5 sampleGaussian()=28935.5
Settings for the state space 'Space2'
  - state validity check resolution: 5%
  - valid segme